In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from wordcloud import WordCloud, STOPWORDS
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import word_tokenize
from langdetect import detect, DetectorFactory
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

In [2]:
input = '../data/data_intermediaire_train.parquet'

In [3]:
df = pd.read_parquet(input)
df.iloc[0]['text']

'The Sea Is Watching has been made from an original Akira Kurosawa script, and it is indeed a lush and warm film. Watching it will be a pleasure !<br /><br />Kei Kumai as director is certainly no equal to the old but everlasting master (particularly the mass scenes in the beginning of the film has some terrible acting), but the overall mood and scenery is very enjoyable. Another thing that is missed here: Kurosawa always managed to let the characters be so much more then what they are actually showing and doing.<br /><br />Probably that was his magic on set while shooting; and just maybe this script was not fully up to par yet.<br /><br />Maybe we just miss the eye of the master.<br /><br />This is one lovely and sweet film, but it is no Kurosawa. To expect that might well be very silly...'

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
df_vect = vectorizer.fit_transform(df['text'])

In [4]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer

# Vectorisation des documents
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['text'])

# Réduction de la dimensionnalité avec TruncatedSVD (similaire à LSA)
svd = TruncatedSVD(n_components=50)
X_reduced = svd.fit_transform(X)

# Apprentissage des embeddings de mots
vocab = vectorizer.get_feature_names_out()
word_vectors = svd.components_.T

# Apprentissage du modèle de sentiment
model = LogisticRegression()
model.fit(X_reduced, df['label'])

# Fonction pour obtenir le vecteur d'un mot
def get_word_vector(word):
    if word in vocab:
        idx = np.where(vocab == word)[0][0]
        return word_vectors[idx]
    else:
        return None


# Prédiction de sentiment pour un nouveau document
new_document = ["this movie is incredible"]
new_document_vec = vectorizer.transform(new_document)
new_document_reduced = svd.transform(new_document_vec)
predicted_sentiment = model.predict(new_document_reduced)
print(f"Sentiment prédit pour le nouveau document: {predicted_sentiment[0]}")


Embedding pour le mot 'incroyable': None
Sentiment prédit pour le nouveau document: 0


In [ ]:
df_vect

In [ ]:
# Charger les données
file_path = '../data/data_intermediaire_train.parquet'
df = pd.read_parquet(file_path)

# Ajouter des stopwords supplémentaires si nécessaire
additional_stopwords = set(["movie","film"])
stopwords = STOPWORDS.union(additional_stopwords)

# Créer un nuage de mots pour chaque catégorie (0: mauvais film, 1: bon film)
for label, category in [(0, "Mauvais film"), (1, "Bon film")]:
    # Filtrer les données pour la catégorie actuelle
    category_data = df[df['label'] == label]['text'].str.cat(sep=' ')

    # Générer le nuage de mots
    wordcloud = WordCloud(stopwords=stopwords, background_color='white', width=800, height=400).generate(category_data)

    # Afficher le nuage de mots
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(f"Word Cloud for {category}")
    plt.show()


In [ ]:
nltk.download('punkt')

DetectorFactory.seed = 0  
def detect_language(text):
    try:
        return detect(text)
    except:
        return 'unknown'

df['language'] = df['text'].apply(detect_language)

language_counts = df['language'].value_counts()
print("Distribution des langues détectées :")
print(language_counts)

In [ ]:
df[df.language=='nl']

In [ ]:
def count_words(text):
    if isinstance(text, str):  
        words = text.split()  
        return len(words)
    return 0

# Ajouter une colonne pour le nombre de mots
df['word_count'] = df['text'].apply(count_words)

# Statistiques sur le nombre de mots
word_count_distribution = df['word_count'].describe()
print("\nStatistiques sur le nombre de mots :")
print(word_count_distribution)

# Visualisation de la distribution du nombre de mots
plt.figure(figsize=(10, 6))
plt.hist(df['word_count'], bins=50, alpha=0.7, color='blue')
plt.title('Distribution du nombre de mots par revue')
plt.xlabel('Nombre de mots')
plt.ylabel('Fréquence')
plt.grid(True)
plt.show()